* most purchases are made by males, accounting for 83% of purchases. However, the average spent by females is greater than males
	
* the 20 to 24 age group makes the greatest absolute number of purchases and spends the most in aggerate

* the "Final Critic" is the most purchased item producing the highest revenue  



In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import os

In [3]:
#load file and convert to data frame

csv_path = os.path.join("Resources", "purchase_data.csv")
game_dt = pd.read_csv(csv_path)
game_dt.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#check the data and clean if needed
game_dt.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [5]:
#get total players by getting list of unique usernames and then getting length of list 

total_players = len(game_dt["SN"].unique())
total_players_df = pd.DataFrame({"Total Players": [total_players]})
total_players_df

,Total Players
0,576


In [6]:
#using various panda functions .unique, .mean, .count and .sum to get the required data points 


unique_items = len(game_dt["Item ID"].unique())
avg_purchase_price = game_dt["Price"].mean()
total_purchases = game_dt["Purchase ID"].count()
total_revenue = game_dt["Price"].sum()
purchasing_analysis_df = pd.DataFrame ({"Number of Unique Items" : [unique_items], "Average Price": [avg_purchase_price], 
                                        "Number of Purchases" : [total_purchases], "Total Revenue" : [total_revenue] })
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [7]:
#using .value_counts on gender column to get the count of each gender 

gender = game_dt["Gender"].value_counts()
gender_df = pd.DataFrame(gender)
gender_df["Percentage"] = (gender_df["Gender"]/gender_df["Gender"].sum())*100
gender_df

,Gender,Percentage
Male,652,83.589744
Female,113,14.487179
Other / Non-Disclosed,15,1.923077


In [8]:
# getting each required data column into a separate serious using .groupby them merging them into one data frame by using .merge


gender_grouped = game_dt.groupby("Gender")
gender_count =gender_grouped.count().loc[:, "Price"]
gender_average_price = gender_grouped.mean().loc[:, "Price"]
gender_total_revenue = gender_grouped.sum().loc[:, "Price"]

final_gender_df = pd.merge(gender_count, gender_average_price, on="Gender")
final_gender_df = pd.merge(final_gender_df, gender_total_revenue, on="Gender")

final_gender_df = final_gender_df.rename(columns= {"Price_x": "Purchase Count", "Price_y" : "Average Purchase Price",
                                                   "Price" : "Total Purchase Value"})

# to get Avg Total Purchase per Person for each gender, first creating a separate data frame for each gender, 
# then grouping by users to aggerate multiple purchases, then applying .mean on the price column to get average 

                                       
male_game_dt = game_dt.loc[game_dt["Gender"] == "Male", :]
female_game_dt = game_dt.loc[game_dt["Gender"] == "Female", :]
other_game_dt = game_dt.loc[game_dt["Gender"] == "Other / Non-Disclosed", :]

avg_total_purchase_male = male_game_dt.groupby("SN").sum()["Price"].mean()
avg_total_purchase_female = female_game_dt.groupby("SN").sum()["Price"].mean()
avg_total_purchase_other = other_game_dt.groupby("SN").sum()["Price"].mean()

avg_total_purchases_final_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"], 
                             "Avg Total Purchase per Person" : [avg_total_purchase_male, avg_total_purchase_female, avg_total_purchase_other]})


final_gender_df = pd.merge(final_gender_df, avg_total_purchases_final_df, on="Gender")

final_gender_df



,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,3.203009,361.94,4.468395
1,Male,652,3.017853,1967.64,4.065372
2,Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [9]:
# defining bins and group name then using cut to insert buckets column into data frame 
# then grouping the data by the buckets and using .agg to apply multiple functions  (count, sum. mean) on the grouped df



bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", " 40+"]

game_dt ["Age Group"] = pd.cut(game_dt["Age"], bins, labels= group_names, include_lowest=True)

age_grouped = game_dt.groupby("Age Group")

age_grouped_dt = age_grouped["Price"].agg(["count", "mean", "sum"])

age_grouped_dt = age_grouped_dt.rename(columns= {"count" : "Purchase Count",
                                                            "sum" : "Total Purchase Value", 
                                                            "mean" : "Averge Purchase Price"})


# to get Avg Total Purchase per Person first creating a df grouped by username,
# then getting age group bucket by uning .max on the grouped df's "Age Group" columns
# then merging the two df's together by username then grouping again by "age group" and applying  .mean to get average 




username_grouped = game_dt.groupby(["SN"])

username_grouped_total_purchase = username_grouped.sum()

username_grouped_total_purchase_dt = pd.DataFrame(username_grouped_total_purchase)

username_grouped_total_purchase_dt = username_grouped_total_purchase_dt.reset_index()


username_grouped_age_dt = username_grouped["Age Group"].max()

username_grouped_age_dt = pd.DataFrame(username_grouped_age_dt)

username_grouped_age_dt= username_grouped_age_dt.reset_index()


age_username_merged_dt =pd.merge(username_grouped_age_dt, username_grouped_total_purchase_dt, on="SN")


age_grouped2 = age_username_merged_dt.groupby("Age Group")
age_grouped2_average_purchase = age_grouped2["Price"].mean()

age_grouped2_average_purchase_dt = pd.DataFrame(age_grouped2_average_purchase)

age_grouped2_average_purchase_dt = age_grouped2_average_purchase_dt.reset_index()


age_group_final_dt = pd.merge(age_grouped_dt,age_grouped2_average_purchase_dt, on="Age Group")

age_group_final_dt = age_group_final_dt.rename(columns={"Price" : "Avg Total Purchase per Person"})

age_group_final_dt

,Age Group,Purchase Count,Averge Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<10,23,3.353478,77.13,4.537059
1,10-14,28,2.956429,82.78,3.762727
2,15-19,136,3.035956,412.89,3.858785
3,20-24,365,3.052219,1114.06,4.318062
4,25-29,101,2.900990,293.00,3.805195
5,30-34,73,2.931507,214.00,4.115385
6,35-39,41,3.601707,147.67,4.763548
7,40+,13,2.941538,38.24,3.186667


In [10]:
# using the groupby function to get the various data points


username_grouped_dt = pd.DataFrame(username_grouped["Price"].agg(["count","mean", "sum"]))

username_grouped_dt= username_grouped_dt.rename(columns={"count" : "Purchase Count",
                                                        "mean" : "Average Purchase Price",
                                                        "sum" : "Total Purchase Value"})



username_grouped_sorted_purchases_value_dt = username_grouped_dt.sort_values("Total Purchase Value",ascending=False)
username_grouped_sorted_purchases_value_dt.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [11]:
#using groupby function to get the data point then using .sort_values


item_grouped = game_dt.groupby(["Item Name"])

item_grouped_dt = pd.DataFrame(item_grouped["Price"].agg(["count","max", "sum"]))
item_grouped_dt= item_grouped_dt.rename(columns={"count" : "Purchase Count",
                                                        "max" : "Item Price",
                                                        "sum" : "Total Purchase Value"})

item_grouped_item_id = pd.DataFrame(item_grouped["Item ID"].max())

item_grouped_item_id_dt = pd.DataFrame(item_grouped_item_id)

item_grouped_item_id_dt = item_grouped_item_id_dt.reset_index()


item_grouped_dt = pd.merge(item_grouped_dt, item_grouped_item_id_dt, on= "Item Name")

item_grouped_dt = item_grouped_dt[["Item ID", "Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]

item_grouped_dt

item_grouped_sorted_purchasecount_dt = item_grouped_dt.sort_values("Purchase Count",ascending=False)
item_grouped_sorted_purchasecount_dt.head(10)

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
56,92,Final Critic,13,4.88,59.99
93,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
98,132,Persuasion,9,3.33,28.99
92,82,Nirvana,9,4.90,44.10
51,108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
55,145,Fiery Glass Crusader,9,4.58,41.22
105,19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16
23,75,Brutality Ivory Warmace,8,2.42,19.36
125,103,Singed Scalpel,8,4.35,34.80
122,37,"Shadow Strike, Glory of Ending Hope",8,3.16,25.28


In [12]:
# using .sort_values to resort by "Total Purchase Value" column

item_grouped_sorted_purchasecount_dt = item_grouped_dt.sort_values("Total Purchase Value", ascending=False)
item_grouped_sorted_purchasecount_dt.head()

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
56,92,Final Critic,13,4.88,59.99
93,178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
92,82,Nirvana,9,4.90,44.10
55,145,Fiery Glass Crusader,9,4.58,41.22
125,103,Singed Scalpel,8,4.35,34.80
